- เพิ่มงานใหม่: ผู้ใช้สามารถเพิ่มงานโดยมีชื่อ, คำอธิบาย, และวันที่ครบกำหนด โดยแต่ละงานควรมีรหัสเฉพาะ (ID) ของตัวเอง
- ดูงานทั้งหมด: แสดงงานทั้งหมดในรูปแบบที่มีโครงสร้าง โดยแยกงานออกเป็นประเภท "รอดำเนินการ" และ "เสร็จสิ้น"
- ทำเครื่องหมายว่างานเสร็จสิ้น: มีฟังก์ชันให้ผู้ใช้ทำเครื่องหมายว่างานเสร็จสิ้น โดยอ้างอิงจากรหัสเฉพาะ (ID) ของงาน
- ลบงาน: ผู้ใช้สามารถลบงานโดยใช้รหัสเฉพาะ (ID)
- บันทึกและโหลดงาน: งานทั้งหมดควรถูกบันทึกลงในไฟล์ JSON และโหลดกลับเมื่อโปรแกรมเริ่มต้น
- ค้นหางาน: เพิ่มฟังก์ชันสำหรับการค้นหางานโดยใช้คำสำคัญหรือวันที่ครบกำหนด

In [ ]:
import uuid
import datetime as dt
import json

class TaskManager:
    status_Inprogress = "Inprogress"
    status_Done = "Done"

    def __init__(self):
        self.taskList = self.load_task_list() #Load task list from Json

    #Find task index using ID
    def findTaskIndex(self, _taskID):
        for index, task in enumerate(self.taskList):
            if task[0] == _taskID:
                return index
        print(f"Not found task: {_taskID}")
        print("====================================================================")
        return None

    #Random 6 digit ID
    def randomTaskID(self):
        rand_id = str(uuid.uuid4().int)[:6]
        return rand_id

    #Show task infomation
    def printTask(self, _task):
        object_dt = _task[3]
        print(f"""ID: {_task[0]}
Name: {_task[1]}
Description: {_task[2]}
Deadline: {object_dt}
Status: {_task[4]}
        """)
        
    #Add a task to list and save
    def addTask(self, _name, _description, _day, _month, _year):
        try:
            _day = int(_day)
            _month = int(_month)
            _year = int(_year)
            # สร้างวันที่
            valid_date = dt.date(_year, _month, _day)
        except ValueError as e:
            print(f"Error: {e}")
            self.mainSection()
            return None
        
        task_id = self.randomTaskID()
        new_task = [task_id, _name, _description, valid_date, self.status_Inprogress]
        self.taskList.append(new_task)  # Add task to taskList
        print("\n---Added task success---")
        self.printTask(new_task)  #Show task infomation
        print("====================================================================")
        self.save_task_list(self.taskList) #save task
        self.mainSection() #back to main section

    #Show all task infomation in list
    def viewAllTask(self):
        if not self.taskList:
            print("\nNo tasks available.\n")
        else:
            print("\n-----All Tasks-----")
            _inprogressList = []
            _doneList = []
            for task in self.taskList:
                if task[4] == self.status_Inprogress:
                    _inprogressList.append(task)
                elif task[4] == self.status_Done:
                    _doneList.append(task)

            print("---Inprogress Task---")
            for task in _inprogressList:
                self.printTask(task)
            print("---Done Task---")
            for task in _doneList:
                self.printTask(task)
        print("====================================================================")
        self.mainSection() #back to main section
        

    #Change task status from "Inprogress" to "Done"
    def checkTaskToDone(self, _taskID):
        _index = self.findTaskIndex(_taskID)
        if _index is not None:
            task = self.taskList[_index]
            task[4] = self.status_Done
            print(f"Change task's status {_taskID}: {self.status_Inprogress} ---> {self.status_Done}")
        print("====================================================================")
        self.save_task_list(self.taskList) #save task
        self.mainSection() #back to main section

    #Delete task by ID
    def deleteTask(self, _taskID):
        _index = self.findTaskIndex(_taskID)
        if _index is not None:
            removed_task = self.taskList.pop(_index)
            print(f"Task removed: {removed_task[0]} - {removed_task[1]}\n")
        
        print("====================================================================")
        self.save_task_list(self.taskList) #save task
        self.mainSection() #back to main section

    # show task that must be completed before the deadline
    def searchTodoTaskBeforeDeadline(self, _day, _month, _year):
        try:
            _day = int(_day)
            _month = int(_month)
            _year = int(_year)
            # สร้างวันที่
            dt_input = dt.date(_year, _month, _day)
            dt_now = dt.date.today()
        except ValueError as e:
            print(f"Error: {e}")
            self.mainSection()
            return None
        print(f"\nTasks that must be completed before {dt_input}:")
        if not self.taskList:
            print("\nNo tasks available.\n")
        else:
            for task in self.taskList:
                dt_dead = task[3]
                difference = dt_input - dt_dead
                if difference.days >= 0:
                    _remainTime = dt_dead - dt_now
                    print(f"Remaining time: {_remainTime.days} days")
                    self.printTask(task)
        print("====================================================================")
        self.mainSection()
        

    # Function to save taskList as JSON file.
    def save_task_list(self, data, file_name="task_list.json"):
        # แปลงวันที่ (datetime.date) เป็น string
        for task in data:
            if isinstance(task[3], dt.date):
                task[3] = task[3].isoformat()  # Convert to ISO 8601 string

        with open(file_name, "w", encoding="utf-8") as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
        print("Task's list is saved")
        print("====================================================================")

    # Function to load taskList from JSON file
    def load_task_list(self, file_name="task_list.json"):
        try:
            with open(file_name, "r", encoding="utf-8") as file:
                data = json.load(file)

            #Convert date (string) back to datetime.date.
            for task in data:
                if isinstance(task[3], str):
                    task[3] = dt.date.fromisoformat(task[3])

            return data
        except (FileNotFoundError, json.JSONDecodeError):
            print(f"File '{file_name}' Not found or corrupted. Start with empty taskList.")
            return []

    # main application
    def mainSection(self):
        print("<<<Main section>>>")
        while True:
            _command = input("Input command: ") # Input fill command

            # command handler
            if _command == "command":
                print("""
---- All Commands ----
1. command - Display a list of all available commands.  
2. add - Add a new task by specifying the name, description, and deadline.  
3. view - Show all tasks in the system along with their status (In Progress or Done).  
4. done - Mark a task as "Done" by providing its unique ID.  
5. delete - Remove a task from the system by using its unique ID.  
6. search - Find and display tasks that need to be completed before a specific deadline.  
7. exit - Exit the program.  
                """)
                
            elif _command == "exit":
                self.save_task_list(self.taskList)
                print("Program is closed")
                break
            elif _command == "add":
                _name = input("Task's name: ").strip()
                _description = input("Task's description: ").strip()
                print("Input task's deadline, Please fill in the deadline information as numbers.")
                _day = int(input("Day: "))
                _month = int(input("Month: "))
                _year = int(input("Year: "))
                self.addTask(_name, _description, _day, _month, _year)
                break
            elif _command == "view":
                self.viewAllTask()
                break
            elif _command == "done":
                _taskID = input("Task's id.")
                self.checkTaskToDone(_taskID)
                break
            elif _command == "delete":
                _taskID = input("Task you want to delete.")
                self.deleteTask(_taskID)
                break
            elif _command == "search":
                print("Input a deadline to find tasks that need to be completed first.")
                _day = int(input("Day: "))
                _month = int(input("Month: "))
                _year = int(input("Year: "))
                self.searchTodoTaskBeforeDeadline(_day, _month, _year)
                break
            else:
                print("error: No command")
            
app = TaskManager()
app.mainSection() # Init application